### **ORJINAL KOD**

In [40]:
# Gerekli kütüphaneleri içe aktar
import os
import cv2
import numpy as np
from scipy.stats import kurtosis, skew
from collections import Counter
from google.colab.patches import cv2_imshow  # Colab için görüntü gösterme

# 1. Klasörden görüntüleri yükle ve 128x128 boyutunda gri tonlamaya dönüştür
def load_images_from_folder(folder_path):
    images = []
    labels = []
    label_mapping = {"camera": 1, "flamingo": 2, "pizza": 3}  # Sınıf etiketleri

    for label_name in os.listdir(folder_path):  # Her alt klasör için döngü
        class_path = os.path.join(folder_path, label_name)
        if not os.path.isdir(class_path):
            continue  # Dosya değilse atla

        for filename in os.listdir(class_path):  # Alt klasördeki her görüntü
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Gri tonlamaya çevir ve 128x128 boyutuna getir
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                resized_img = cv2.resize(gray_img, (128, 128))
                # Görüntünün türevini al (Sobel filtresi)
                sobelx = cv2.Sobel(resized_img, cv2.CV_64F, 1, 0, ksize=5)
                sobely = cv2.Sobel(resized_img, cv2.CV_64F, 0, 1, ksize=5)
                gradient = np.sqrt(sobelx**2 + sobely**2)
                # 128x128 görüntüyü 1x16384 vektörüne dönüştür
                gradient_vector = gradient.flatten().astype(np.float32)  # Değişiklik burada
                images.append(gradient_vector)
                labels.append(label_mapping[label_name])  # Sınıf etiketini ekle

    return np.array(images), np.array(labels)

# Eğitim ve test verilerini yükle
x_train, y_train = load_images_from_folder('/content/train')
x_test, y_test = load_images_from_folder('/content/test')

# 2. Vektörlerden 6 özellik çıkar (Feature Extraction)
def extract_features(vectors):
    features = []
    for vector in vectors:
        # Çarpıklık (Skewness)
        skewness = skew(vector)
        # Basıklık (Kurtosis)
        kurt = kurtosis(vector)
        # Tepe Faktörü (Crest Factor)
        crest_factor = np.max(np.abs(vector)) / np.sqrt(np.mean(np.square(vector))) if np.mean(np.square(vector)) != 0 else 0
        # Şekil Faktörü (Shape Factor)
        shape_factor = np.sqrt(np.mean(np.square(vector))) / np.mean(np.abs(vector)) if np.mean(np.abs(vector)) != 0 else 0
        # Darbe Faktörü (Impulse Factor)
        impulse_factor = np.max(np.abs(vector)) / np.mean(np.abs(vector)) if np.mean(np.abs(vector)) != 0 else 0
        # Marj Faktörü (Margin Factor)
        margin_factor = np.max(np.abs(vector)) / np.median(np.abs(vector)) if np.median(np.abs(vector)) != 0 else 0
        # Özellikleri bir listeye ekle
        features.append([skewness, kurt, crest_factor, shape_factor, impulse_factor, margin_factor])
    return np.array(features)

# Eğitim ve test özelliklerini çıkar
x_train_features = extract_features(x_train)
x_test_features = extract_features(x_test)

# 3. KNN Fonksiyonunu Tanımla
def KNN(x_train, y_train, sample_test, k=5):
    # Öklid mesafesini hesapla
    distances = np.linalg.norm(x_train - sample_test, axis=1)
    # Mesafelere göre sıralayıp en yakın k komşuyu seç
    nearest_neighbors = np.argsort(distances)[:k]
    # En sık görülen sınıfı bul ve döndür
    top_class = Counter(y_train[nearest_neighbors]).most_common(1)[0][0]
    return top_class

# 4. Modelin Test Edilmesi ve Tahmin Sonuçlarının Gösterilmesi
correct_predictions = 0  # Doğru tahmin sayısını tutmak için
class_mapping = {1: "camera", 2: "flamingo", 3: "pizza"}  # Sınıf isimleri

print(f"{'Test Image':<12} {'Predicted Class':<15} {'Actual Class':<15}")

for i in range(len(x_test_features)):
    # Her test görüntüsü için tahmin yap
    prediction = KNN(x_train_features, y_train, x_test_features[i], k=5)
    actual_label = y_test[i]  # Gerçek etiket

    # Tahmin edilen ve gerçek sınıfı ekrana yazdır
    print(f"Image {i+1:<7} {class_mapping[prediction]:<15} {class_mapping[actual_label]:<15}")

    if prediction == actual_label:
        correct_predictions += 1  # Doğru tahmin durumunda artır

# Doğruluk oranını hesapla
accuracy = correct_predictions / len(y_test) * 100  # 'y_test_features' yerine 'y_test' kullanıldı
print(f"\nTest Doğruluğu: {accuracy:.2f}%")


Test Image   Predicted Class Actual Class   
Image 1       camera          camera         
Image 2       flamingo        camera         
Image 3       flamingo        camera         
Image 4       flamingo        camera         
Image 5       flamingo        camera         
Image 6       flamingo        flamingo       
Image 7       flamingo        flamingo       
Image 8       flamingo        flamingo       
Image 9       flamingo        flamingo       
Image 10      flamingo        flamingo       
Image 11      flamingo        flamingo       
Image 12      camera          flamingo       
Image 13      pizza           flamingo       
Image 14      pizza           pizza          
Image 15      pizza           pizza          
Image 16      pizza           pizza          
Image 17      flamingo        pizza          
Image 18      flamingo        pizza          
Image 19      pizza           pizza          

Test Doğruluğu: 57.89%


### **INSTALL KERAS**

In [17]:
!pip install keras
!pip install keras.preprocessing


### **KNN DE DÜZENLENME YAPILMIŞ VE KERAS KULLANILMIŞ HALİ**

In [30]:
# Gerekli kütüphaneleri içe aktar
import os
import cv2
import numpy as np
from scipy.stats import kurtosis, skew
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # TensorFlow Keras kullan

# 1. Klasörden görüntüleri yükle ve 128x128 boyutunda gri tonlamaya dönüştür
def load_images_from_folder(folder_path):
    images = []
    labels = []
    label_mapping = {"camera": 1, "flamingo": 2, "pizza": 3}  # Sınıf etiketleri

    for label_name in os.listdir(folder_path):  # Her alt klasör için döngü
        class_path = os.path.join(folder_path, label_name)
        if not os.path.isdir(class_path):
            continue  # Dosya değilse atla

        for filename in os.listdir(class_path):  # Alt klasördeki her görüntü
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Gri tonlamaya çevir ve 128x128 boyutuna getir
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                resized_img = cv2.resize(gray_img, (128, 128))
                images.append(resized_img)  # Gri görüntüyü ekle
                labels.append(label_mapping[label_name])  # Sınıf etiketini ekle

    return np.array(images), np.array(labels)

# Eğitim ve test verilerini yükle
x_train, y_train = load_images_from_folder('/content/train')
x_test, y_test = load_images_from_folder('/content/test')

# 2. Görüntüleri augment etme
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augment edilen eğitim verilerini saklayacak yer
augmented_images = []
augmented_labels = []

# Eğitim verilerini augment et
for img, label in zip(x_train, y_train):
    img = img.reshape((1,) + img.shape + (1,))  # Şekil değiştirin (128, 128, 1)
    for batch in datagen.flow(img, batch_size=1):
        augmented_images.append(batch[0].reshape(128, 128))  # Augmented görüntüyü ekle
        augmented_labels.append(label)  # Etiketi ekle
        if len(augmented_images) >= len(x_train) * 2:  # Orijinal veri sayısının iki katı kadar
            break  # Döngüyü durdur

augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# 3. Vektörlerden 6 özellik çıkar (Feature Extraction)
def extract_features(vectors):
    features = []
    for vector in vectors:
        # Çarpıklık (Skewness)
        skewness = skew(vector.flatten())
        # Basıklık (Kurtosis)
        kurt = kurtosis(vector.flatten())
        # Tepe Faktörü (Crest Factor)
        crest_factor = np.max(np.abs(vector)) / np.sqrt(np.mean(np.square(vector))) if np.mean(np.square(vector)) != 0 else 0
        # Şekil Faktörü (Shape Factor)
        shape_factor = np.sqrt(np.mean(np.square(vector))) / np.mean(np.abs(vector)) if np.mean(np.abs(vector)) != 0 else 0
        # Darbe Faktörü (Impulse Factor)
        impulse_factor = np.max(np.abs(vector)) / np.mean(np.abs(vector)) if np.mean(np.abs(vector)) != 0 else 0
        # Marj Faktörü (Margin Factor)
        margin_factor = np.max(np.abs(vector)) / np.median(np.abs(vector)) if np.median(np.abs(vector)) != 0 else 0
        # Özellikleri bir listeye ekle
        features.append([skewness, kurt, crest_factor, shape_factor, impulse_factor, margin_factor])
    return np.array(features)

# Özellikleri çıkar
x_train_features = extract_features(augmented_images)
x_test_features = extract_features(x_test)

# 4. KNN Fonksiyonunu Tanımla
def KNN(x_train, y_train, sample_test, k=5):
    # Öklid mesafesini hesapla
    distances = np.linalg.norm(x_train - sample_test, axis=1)
    # Mesafelere göre sıralayıp en yakın k komşuyu seç
    nearest_neighbors = np.argsort(distances)[:k]
    # En sık görülen sınıfı bul ve döndür
    top_class = Counter(y_train[nearest_neighbors]).most_common(1)[0][0]
    return top_class

# 5. Modelin Test Edilmesi ve Tahmin Sonuçlarının Gösterilmesi
def evaluate_model(x_test_features, y_test, x_train_features, y_train):
    correct_predictions = 0  # Doğru tahmin sayısını tutmak için
    class_mapping = {1: "camera", 2: "flamingo", 3: "pizza"}  # Sınıf isimleri

    print(f"{'Test Image':<12} {'Predicted Class':<15} {'Actual Class':<15}")

    for i in range(len(x_test_features)):
        # Her test görüntüsü için tahmin yap
        prediction = KNN(x_train_features, y_train, x_test_features[i], k=5)
        actual_label = y_test[i]  # Gerçek etiket

        # Tahmin edilen ve gerçek sınıfı ekrana yazdır
        print(f"Image {i+1:<7} {class_mapping[prediction]:<15} {class_mapping[actual_label]:<15}")

        if prediction == actual_label:
            correct_predictions += 1  # Doğru tahmin durumunda artır

    # Doğruluk oranını hesapla
    accuracy = correct_predictions / len(y_test) * 100
    print(f"\nTest Doğruluğu: {accuracy:.2f}%")

# Modeli değerlendir
evaluate_model(x_test_features, y_test, x_train_features, augmented_labels)


Test Image   Predicted Class Actual Class   
Image 1       flamingo        camera         
Image 2       flamingo        camera         
Image 3       flamingo        camera         
Image 4       flamingo        camera         
Image 5       flamingo        camera         
Image 6       flamingo        flamingo       
Image 7       flamingo        flamingo       
Image 8       flamingo        flamingo       
Image 9       flamingo        flamingo       
Image 10      flamingo        flamingo       
Image 11      flamingo        flamingo       
Image 12      flamingo        flamingo       
Image 13      flamingo        flamingo       
Image 14      flamingo        pizza          
Image 15      flamingo        pizza          
Image 16      flamingo        pizza          
Image 17      flamingo        pizza          
Image 18      flamingo        pizza          
Image 19      flamingo        pizza          

Test Doğruluğu: 42.11%
